# Ejercicios

Construye la red de retweets de #RepealThe19th, un controvertido hashtag utilizado durante la campaña presidencial estadounidense de 2016 para abogar por la derogación de la 19ª enmienda de la Constitución estadounidense, que concede a las mujeres el derecho al voto . Los datos para este ejercicio se encuentran en el directorio datasets. El archivo llamado repealthe19th.jsonl.gz incluye:

- 23.343 tuits que contienen el hashtag. Cada línea es un objeto JSON de tuit. Después de analizar el archivo, debe verificar que tiene esta cantidad de tweets; una desviación de este número indicaría errores de análisis que podrían afectar sus respuestas.

Ten en cuenta lo siguiente en : 
- (i) La dirección de los enlaces sigue el flujo de información: si Alicia retuitea a Bob, existe un enlace de Bob a Alicia. 
- (ii) Elimina los auto-bucles; puedes hacerlo después de crear la red, o modificar tu código de creación de red para no añadirlos en absoluto.

Y con la red creada, responde las siguientes preguntas:

1. ¿Cuántos nodos hay en la red retweet?
2. ¿Cuántos enlaces hay en la red retweet?
3. ¿Cuál es el nombre de pantalla del nodo con mayor out-strength en la red? ¿Cuál es su out-strength? (La fuerza de salida indica cuánto se ha retuiteado a un usuario (influencia), mientras que la fuerza de entrada muestra cuántos retuits ha hecho (actividad)).
4. ¿Cuál es el nombre de pantalla del nodo con la segunda mayor fuerza de salida de la red?
sobre su comportamiento en Internet.
5. ¿Cuál es el ID del tuit más retuiteado? Utilice el atributo id str; cuando se trabaja con archivos JSON, esta suele ser una buena práctica debido a que los ID de los tweets son de 64 bits. 
6. ¿Cuántos nodos de la red de retweets tienen fuerza de salida cero?
7. ¿Cuál de las siguientes opciones describe mejor la conectividad de esta red?

        a.Fuertemente conectada
        b.Débilmente conectada
        c.Conectada
        d.Desconectada
8.  ¿Cuántos nodos hay en el mayor componente débilmente conectado de esta red?

In [2]:
import networkx as nx
import json
import gzip
from collections import defaultdict
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
retweet_edges = []  
tweet_counts = defaultdict(int)  
most_retweeted = {'id_str': None, 'retweet_count': -1}  

In [4]:
with gzip.open('repealthe19th.jsonl.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            tweet = json.loads(line)  
            tweet_counts['total'] += 1 
            
            if 'retweeted_status' in tweet:
                source = tweet['retweeted_status']['user']['screen_name']
                target = tweet['user']['screen_name']
                
                if source != target:
                    retweet_edges.append((source, target))
                
                if tweet['retweeted_status']['retweet_count'] > most_retweeted['retweet_count']:
                    most_retweeted = {
                        'id_str': tweet['retweeted_status']['id_str'],
                        'retweet_count': tweet['retweeted_status']['retweet_count']
                    }
            
        except json.JSONDecodeError:
            continue
        
print(f"Total de tweets procesados: {tweet_counts['total']}")
print(f"Relaciones de retweet encontradas: {len(retweet_edges)}")

Total de tweets procesados: 23343
Relaciones de retweet encontradas: 14834


In [5]:
G = nx.DiGraph()  
G.add_edges_from(retweet_edges)  

In [6]:
print(f"Cantidad de nodos en la red retweet: {G.number_of_nodes()}.")
print(f"Cantidad de enlaces en la red retweet: {G.number_of_edges()}.")


Cantidad de nodos en la red retweet: 14862.
Cantidad de enlaces en la red retweet: 14795.


In [7]:
out_strength = dict(G.out_degree(weight='weight'))
sorted_out_strength = sorted(out_strength.items(), key=lambda x: x[1], reverse=True)

top_user, top_strength = sorted_out_strength[0]

print(f"Usuario más influyente (mayor out-strength): {top_user}")
print(f"Número de retweets recibidos (out-strength): {top_strength}")

Usuario más influyente (mayor out-strength): Samandjunk
Número de retweets recibidos (out-strength): 900


In [8]:
top_user, top_strength = sorted_out_strength[1]

print(f"Segundo usuario más influyente (mayor out-strength): {top_user}")
print(f"Número de retweets recibidos (out-strength): {top_strength}")

Segundo usuario más influyente (mayor out-strength): m_kaish_esq
Número de retweets recibidos (out-strength): 394


In [9]:
print(f"ID del tuit más retuiteado: {most_retweeted['id_str']}")

ID del tuit más retuiteado: 786335044981907456


In [10]:
zero_out_strength = sum(1 for node in G.nodes() if out_strength.get(node, 0) == 0)

print(f"Cantidad de nodos con out-strength cero: {zero_out_strength}")

Cantidad de nodos con out-strength cero: 12916


In [19]:
if nx.is_strongly_connected(G):
    connectivity = "Fuertemente conectada"
elif  nx.is_weakly_connected(G):
    connectivity = "Débilmente conectada"
else:
    connectivity = "Desconectada"

print(f"Conectividad de la red: {connectivity}.")

Conectividad de la red: Desconectada.


In [ ]:
components = list(nx.weakly_connected_components(G))
largest = max(components, key=len)

print(f"Nodos con el mayor componente débilmente conectado: {len(largest)}")

Nodos con el mayor componente débilmente conectado: 11909
